In [1]:
! git clone https://github.com/dungca1512/image_captioning.git

Cloning into 'image_captioning'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 49 (delta 11), reused 35 (delta 3), pack-reused 0
Unpacking objects: 100% (49/49), 7.25 MiB | 6.34 MiB/s, done.


In [2]:
! pip install pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 8.7 MB/s eta 0:00:00:00:01


In [3]:
import csv
from shutil import copyfile
from pycocotools.coco import COCO
from tqdm import tqdm

In [4]:
! mkdir image_captioning/data

In [5]:
!mkdir image_captioning/data/images
!mkdir image_captioning/data/images/train
!mkdir image_captioning/data/images/test

In [6]:
coco = COCO('/kaggle/input/coco2014/captions/annotations/captions_train2014.json')

loading annotations into memory...
Done (t=1.55s)
creating index...
index created!


In [7]:
#get ids of training images
with open('image_captioning/TrainImageIds.csv', 'r') as f:
    reader = csv.reader(f)
    trainIds = list(reader)
    
trainIds = [int(i) for i in trainIds[0]]
print(len(trainIds))

13244


In [8]:
for img_id in tqdm(trainIds):
    path = coco.loadImgs(img_id)[0]['file_name']
    copyfile('/kaggle/input/coco2014/train2014/train2014/' + path, 'image_captioning/data/images/train/' + path)

100%|██████████| 13244/13244 [02:03<00:00, 107.07it/s]


In [9]:
cocoTest = COCO('/kaggle/input/coco2014/captions/annotations/captions_val2014.json')

loading annotations into memory...
Done (t=0.82s)
creating index...
index created!


In [10]:
with open('image_captioning/TestImageIds.csv', 'r') as f:
    reader = csv.reader(f)
    testIds = list(reader)
    
testIds = [int(i) for i in testIds[0]]
print(len(testIds))

3000


In [11]:
for img_id in tqdm(testIds):
    path = cocoTest.loadImgs(img_id)[0]['file_name']
    copyfile('/kaggle/input/coco2014/val2014/val2014/' + path, 'image_captioning/data/images/test/' + path)

100%|██████████| 3000/3000 [00:28<00:00, 106.58it/s]


In [12]:
! python /kaggle/working/image_captioning/build_vocab.py

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
loading annotations into memory...
Done (t=0.97s)
creating index...
index created!
[10000/414113] Tokenized the captions.
[20000/414113] Tokenized the captions.
[30000/414113] Tokenized the captions.
[40000/414113] Tokenized the captions.
[50000/414113] Tokenized the captions.
[60000/414113] Tokenized the captions.
[70000/414113] Tokenized the captions.
[80000/414113] Tokenized the captions.
[90000/414113] Tokenized the captions.
[100000/414113] Tokenized the captions.
[110000/414113] Tokenized the captions.
[120000/414113] Tokenized the captions.
[130000/414113] Tokenized the captions.
[140000/414113] Tokenized the captions.
[150000/414113] Tokenized the captions.
[160000/414113] Tokenized the captions.
[170000/414113] Tokenized the captions.
[180000/414113] Tokenized the captions.
[190000/414113] Tokenized the captions.
[200000/414113] Tokenized the captions.
[210000/41

In [13]:
! python /kaggle/working/image_captioning/train_val_split.py

loading annotations into memory...
Done (t=0.95s)
creating index...
index created!


In [14]:
import gensim
import pickle
from image_captioning.build_vocab import Vocabulary
import numpy as np
import torch

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
model = gensim.models.KeyedVectors.load_word2vec_format('/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True) 

In [16]:
with open('/kaggle/working/image_captioning/data/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
print(len(vocab))

9956


In [17]:
matrix_len = len(vocab.word2idx.keys())
weights_matrix = np.zeros((matrix_len, 300))

for i, word in enumerate(vocab.word2idx.keys()):
    if word in model: 
        weights_matrix[i] = model[word]
    else:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(300, ))

In [18]:
weights = torch.from_numpy(weights_matrix)

In [19]:
torch.save(weights, '/kaggle/working/image_captioning/pretrain_weights.pt')

In [20]:
! python /kaggle/working/image_captioning/train.py --model_name img_cap

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Namespace(model_name='img_cap', model_path='/kaggle/working/image_captioning/models/', crop_size=224, vocab_path='/kaggle/working/image_captioning/data/vocab.pkl', image_dir='/kaggle/working/image_captioning/data/images/train', caption_path='/kaggle/input/coco2014/captions/annotations/captions_train2014.json', log_step=10, save_step=1, val_image_dir='/kaggle/working/image_captioning/data/images/val', val_caption_path='/kaggle/input/coco2014/captions/annotations/captions_val2014.json', embed_size=300, hidden_size=512, num_layers=1, num_epochs=30, batch_size_train=128, batch_size_val=64, num_workers=2, learning_rate=0.001, pretrained=True)
loading annotations into memory...
Done (t=1.06s)
creating index...
index created!
loading annotations into memory...
Done (t=1.11s)
creating index...
index created!
loading annotations into memory...
Done (t=0.87s)
creating index...
inde

In [23]:
! python /kaggle/working/image_captioning/test.py --model_name img_cap

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
loading annotations into memory...
Done (t=0.56s)
creating index...
index created!
loading annotations into memory...
Done (t=0.46s)
creating index...
index created!
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Evaluating Caption: 15010/15011Test Loss : 2.49
Writing captions to json file...
100%|███████

In [24]:
! python /kaggle/working/image_captioning/infer.py --image /kaggle/working/image_captioning/surf.jpg

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<start> a man riding a wave on top of a surfboard . <end>
